In [20]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [21]:
class BatsmanState(TypedDict):

      runs : int
      ball : int
      fours : int
      sixs : int

      strike_rate : float
      boundary_percentage : float
      bpb : float

      summary : str

In [22]:
def strike_rate_function(state : BatsmanState) -> BatsmanState:

      sr = (state['runs'] / state['ball']) * 100

      return {'strike_rate' : sr}

In [23]:
def boundary_percentage_function(state : BatsmanState) -> BatsmanState:

      bp = (((state['fours'] * 4) + (state['sixs'] * 6)) / state['runs']) * 100

      return {'boundary_percentage' : bp}

In [24]:
def bpb_function(state : BatsmanState) -> BatsmanState:

      bpb = state['ball'] / (state['fours'] + state['sixs'])

      return {'bpb' : bpb}

In [25]:
def summary_function(state : BatsmanState) -> BatsmanState:

      summary = f"""
Strike Rate - {state['strike_rate']} \n
Boundary Percentage - {state['boundary_percentage']} \n
Ball per Boundary - {state['bpb']}

"""
      state['summary'] = summary

      return state

In [26]:
# create graph
graph = StateGraph(BatsmanState)

# add node
graph.add_node("strike_rate_function", strike_rate_function)
graph.add_node("boundary_percentage_function", boundary_percentage_function)
graph.add_node("bpb_function", bpb_function)

graph.add_node("summary_function", summary_function)


# add edges
graph.add_edge(START, 'strike_rate_function')
graph.add_edge(START, 'boundary_percentage_function')
graph.add_edge(START, 'bpb_function')

graph.add_edge('strike_rate_function', 'summary_function')
graph.add_edge('boundary_percentage_function', 'summary_function')
graph.add_edge('bpb_function', 'summary_function')

graph.add_edge('summary_function', END)

# compile
workflow = graph.compile()

In [27]:
initial_state = {'runs' : 100, 'ball' : 50, 'fours' : 6, 'sixs' : 4}

final_state = workflow.invoke(initial_state)

In [28]:
final_state

{'runs': 100,
 'ball': 50,
 'fours': 6,
 'sixs': 4,
 'strike_rate': 200.0,
 'boundary_percentage': 48.0,
 'bpb': 5.0,
 'summary': '\nStrike Rate - 200.0 \n\nBoundary Percentage - 48.0 \n\nBall per Boundary - 5.0\n\n'}

In [29]:
final_state['summary']

'\nStrike Rate - 200.0 \n\nBoundary Percentage - 48.0 \n\nBall per Boundary - 5.0\n\n'